In [1]:
!pip3 install tensorflow-rocm

     |████████████████████████████████| 350.8 MB 13 kB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=09c5d16665b8d2fa118967c2cc393ebfb842cd38411d0a3d84f8af4e99316fc7
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68720 sha256=cc2da0782e74162960876b3147f7cebc3ccbde1df685f63e58520cab384f8290
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Successfully uninstalled wrapt-1.13.3
  Attempting uninstall: flatbuffers

In [ ]:
import tensorflow as tf
import tabnet 

In [ ]:
feature_columns = []

In [ ]:
def prepare_features(data, label):
  features = [key for key in data.keys() if key.startswith(feature_columns)]
  return tf.stack[tf.cast(data[key], tf.float32) for key in features], axis=1), tf.cast(label, tf.float32)
def get_dataset(suffix):
  data = tf.data.experimental.make_csv_dataset('/storage/data/dataset_{}.csv'.format(suffix), 
                                              batch_size=1000, label_name='deal_probability', shuffle=True,
                                              shuffle_buffer_size=10000,
                                              num_parallel_reads=tf.data.AUTOTUNE)
  data = data.map(prepare_features, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train = get_dataset('train')
test = get_dataset('test')

In [ ]:
# Use Group Normalization for small batch sizes
model = tabnet.TabNetRegressor(feature_columns=None, 
                               num_regressors=1,
                               num_decision_steps=5, 
                               relaxation_factor=1.2, 
                               sparsity_coefficient=0.001,
                               batch_momentum=0.8,
                               virtual_batch_size=250,
                               feature_dim = 64,
                               output_dim = 32)

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.002, decay_steps=2000, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse', metrics=['mse', 'mae'])

model.fit(train, epochs=15, validation_data=test, verbose=True, steps_per_epoch=10000)

model.summary()

In [ ]:
model.save('/storage/data/tabnet')